In [2]:
import pandas as pd
import re
import unicodedata
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification
import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss



In [1]:
!nvidia-smi


Mon May 12 12:39:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
#prepare data
raw_true = pd.read_csv('dataset/True.csv')
raw_false = pd.read_csv('dataset/Fake.csv')

raw_true["label"] = 1
raw_false["label"] = 0

combined_data = pd.concat([raw_true,raw_false])
combined_data = combined_data.reset_index(drop=True)



In [6]:
#clean data
def clean_text(text):
    if pd.isnull(text) or text.strip() == "":
        return ""
    text =re.sub(r"\s+"," ",text)
    return text
combined_data["text"] = combined_data["text"].apply(clean_text)
#strip the empty text ones
combined_data = combined_data[combined_data["text"].str.strip() != ""]

combined_data["title"] = combined_data["title"].apply(clean_text)
combined_data = combined_data[combined_data["title"].str.strip() != ""]



In [7]:
combined_data["article"] = combined_data["title"] + combined_data["text"]
combined_data = combined_data.drop(["text","title"], axis = 1)


In [ ]:
combined_data

,subject,date,label,article
0,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t..."
1,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...
2,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...
3,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...
4,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...
...,...,...,...,...
44893,Middle-east,"January 16, 2016",0,McPain: John McCain Furious That Iran Treated ...
44894,Middle-east,"January 16, 2016",0,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
44895,Middle-east,"January 15, 2016",0,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
44896,Middle-east,"January 14, 2016",0,How to Blow $700 Million: Al Jazeera America F...


In [ ]:
print(len(combined_data))

44267


In [ ]:
# row = combined_data.loc[8970]

# with open("row_23480.txt", "w", encoding="utf-8") as f:
#     for col in combined_data.columns:
#         f.write(f"{col.upper()}:\n{row[col]}\n\n")

# this is for printing a specific article


In [8]:
#shuffle, frac = 1 means 100% for the file, if axis not given it shuffles the lines
combined_data = combined_data.sample(frac=1).reset_index(drop = True)


In [ ]:
combined_data

,subject,date,label,article
0,News,"May 12, 2017",0,BREAKING: FBI Agents Say White House Will Kil...
1,Middle-east,"May 11, 2016",0,BOILER ROOM – EP #56 – Pharmacological Nightma...
2,left-news,"Jul 1, 2017",0,PRO-TRUMP CHICAGOANS HIT BACK: Use ‘Real Fake’...
3,worldnews,"October 31, 2017",1,China considers three-year jail terms for disr...
4,politicsNews,"June 15, 2016",1,"Trump adds new twist to immigration proposals,..."
...,...,...,...,...
44262,News,"November 8, 2017",0,Sean Hannity Barely Spends 6 Seconds On Last ...
44263,left-news,"Jul 17, 2015",0,SEATTLE MAYOR WANTS TO HELP MUSLIMS FOLLOW SHA...
44264,worldnews,"December 22, 2017",1,"After Reuters arrests, some Myanmar reporters ..."
44265,News,"February 20, 2017",0,Trump Tweets His Dumbest ‘Sweden’ Garbage Yet...


In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

article = combined_data.loc[1,'article']
encoded = tokenizer(article)
print(encoded)
print(encoded["input_ids"])
print(tokenizer.convert_ids_to_tokens(encoded["input_ids"]))  # The actual tokens
print(len(encoded["input_ids"]))  # Number of tokens

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': [101, 8398, 1521, 1055, 3192, 10132, 14456, 2000, 7897, 1010, 4911, 25760, 3513, 5280, 19058, 8398, 1055, 2375, 23890, 2075, 2003, 2763, 2055, 2000, 2131, 1037, 2843, 2062, 1038, 20051, 4630, 2085, 2008, 2002, 1055, 2183, 2000, 2022, 1999, 2436, 1012, 2044, 2035, 1010, 2002, 2074, 2409, 1996, 6396, 2102, 2008, 11274, 2064, 1056, 2031, 9755, 1997, 3037, 1998, 2038, 2525, 2109, 1996, 2436, 1997, 1996, 8798, 2005, 3167, 2449, 1012, 1998, 2004, 2009, 2038, 2074, 2042, 3936, 1010, 1996, 8398, 3192, 10132, 14456, 2000, 7897, 1998, 4911, 25760, 3513, 2055, 2969, 1011, 7149, 1012, 1999, 1996, 2325, 4171, 2709, 2005, 1996, 8398, 3192, 1010, 1996, 3029, 7127, 2027, 4015, 5029, 2000, 1037, 14209, 2711, 1999, 1996, 2197, 4171, 2095, 8398, 1010, 1037, 2266, 1997, 2010, 6234, 2155, 1010, 2030, 3383, 1037, 2449, 3037, 1998, 2036, 5393, 2027, 5117, 1999, 2969, 1011, 7149, 1999, 2627, 2086, 1012, 2013, 2388, 3557, 1024, 6221, 8398, 1055, 11128, 3192, 2038, 4914, 2000, 20084, 2019, 25760, 

In [10]:
#tokenizer(
#     list_of_texts,
#     padding=True,
#     truncation=True,
#     return_tensors="pt",   # or "tf" if you're using TensorFlow
#     max_length=512
# )

articles = combined_data["article"].to_list()
encodings = tokenizer(
    articles,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)


In [ ]:
print(encodings["input_ids"].shape)
input_ids = encodings["input_ids"][0]  # First row
tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(tokens[:50])  # Show first 50 tokens for that article
print(tokenizer.decode(input_ids))


torch.Size([44267, 512])
['[CLS]', 'breaking', ':', 'fbi', 'agents', 'say', 'white', 'house', 'will', 'kill', 'russian', 'investigation', '##age', '##nts', 'within', 'the', 'fbi', 'are', 'worried', 'that', 'president', 'donald', 'trump', 'may', 'succeed', 'in', 'killing', 'the', 'probe', 'into', 'his', 'campaign', 's', 'ties', 'to', 'russia', '.', 'the', 'daily', 'beast', 'is', 'reporting', 'on', 'what', 'has', 'happened', 'at', 'the', 'fbi', 'since']
[CLS] breaking : fbi agents say white house will kill russian investigationagents within the fbi are worried that president donald trump may succeed in killing the probe into his campaign s ties to russia. the daily beast is reporting on what has happened at the fbi since james comey was fired on tuesday. they say that agents who are working on the investigation fear they will be transferred to other departments and that their work will be discontinued. comey, they say, was much more important to the investigation than many, who are outsi

In [11]:
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
labels = combined_data["label"].values

train_input_ids, val_input_ids, train_attention, val_attention, train_labels, val_labels = train_test_split(
    input_ids,
    attention_mask,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)


In [12]:
class MyDataset(Dataset):
    def __init__(self, inputs, masks, labels):
        self.inputs = inputs
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }

train_set = MyDataset(train_input_ids,train_attention,train_labels)
val_set = MyDataset(val_input_ids,val_attention,val_labels)

In [13]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)

In [14]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [16]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [17]:
loss_fn = CrossEntropyLoss()

In [19]:
from google.colab import drive
drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/bert-fake-news.pt"

Mounted at /content/drive


In [20]:

num_epochs = 3
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()

    total_loss = 0

    for batch in train_loader:
        #send the data to either the gpu or cpu
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        #calculate
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        #redo the stepts
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader):.4f}")

    #eval
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"Validation Loss: {val_loss / len(val_loader):.4f}")
    print(f"Validation Accuracy: {correct / total:.4f}")
    val_acc = correct / total

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        #torch.save(model.state_dict(), "bert-fake-news.pt")
        torch.save(model.state_dict(), save_path)
        print(f"Model saved with new best accuracy: {val_acc:.4f}")


Epoch 1: Loss = 0.0103
Validation Loss: 0.0001
Validation Accuracy: 1.0000
✔ Model saved with new best accuracy: 1.0000
Epoch 2: Loss = 0.0013
Validation Loss: 0.0001
Validation Accuracy: 1.0000
Epoch 3: Loss = 0.0007
Validation Loss: 0.0000
Validation Accuracy: 1.0000


In [21]:
from torch.nn.functional import softmax

def predict(text):
    model.eval()
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        predicted = torch.argmax(probs, dim=1).item()

    label = "REAL" if predicted == 1 else "FAKE"
    confidence = probs[0][predicted].item()
    return label, confidence


In [27]:
text = "Donald Trump is against marriage equality. He wants to go back."
label, confidence = predict(text)
print(f"Prediction: {label} ({confidence:.2f} confidence)")


Prediction: FAKE (1.00 confidence)


In [30]:
raw_true_test = pd.read_csv('dataset/True.csv').sample(n=1000, random_state=42)
raw_false_test = pd.read_csv('dataset/Fake.csv').sample(n=1000, random_state=42)

raw_true_test["label"] = 1
raw_false_test["label"] = 0

combined_data_test = pd.concat([raw_true_test,raw_false_test])
combined_data_test = combined_data_test.reset_index(drop=True)

combined_data_test["text"] = combined_data_test["text"].apply(clean_text)
#strip the empty text ones
combined_data_test = combined_data_test[combined_data_test["text"].str.strip() != ""]

combined_data_test["title"] = combined_data_test["title"].apply(clean_text)
combined_data_test = combined_data_test[combined_data_test["title"].str.strip() != ""]

combined_data_test["article"] = combined_data_test["title"] + combined_data_test["text"]
combined_data_test = combined_data_test.drop(["text","title"], axis = 1)


combined_data_test = combined_data_test.sample(frac=1).reset_index(drop = True)


articles_test = combined_data_test["article"].to_list()
encodings_test = tokenizer(
    articles_test,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
print(encodings_test["input_ids"].shape)
input_ids_test = encodings_test["input_ids"]
attention_mask_test = encodings_test["attention_mask"]
labels_test = combined_data_test["label"].values
train_input_ids_test, val_input_ids_test, train_attention_test, val_attention_test, train_labels_test, val_labels_test = train_test_split(
    input_ids_test,
    attention_mask_test,
    labels_test,
    test_size=0.2,
    stratify=labels_test,
    random_state=42
)
train_set_test = MyDataset(train_input_ids_test,train_attention_test,train_labels_test)
val_set_test = MyDataset(val_input_ids_test,val_attention_test,val_labels_test)
train_loader_test = DataLoader(train_set_test, batch_size=16, shuffle=True)
val_loader_test = DataLoader(val_set_test, batch_size=16)
model.eval()
val_loss_test = 0
correct_test = 0
total_test = 0

with torch.no_grad():
    for batch in val_loader_test:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        val_loss_test += loss.item()

        preds = torch.argmax(outputs.logits, dim=1)
        correct_test += (preds == labels).sum().item()
        total_test += labels.size(0)
print(f"Validation Loss: {val_loss_test / len(val_loader_test):.4f}")
print(f"Validation Accuracy: {correct_test / total_test:.4f}")
val_acc_test = correct_test / total_test
print(f"Test Accuracy: {val_acc_test:.4f}")





<ipython-input-30-6a602ecc947d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data_test["title"] = combined_data_test["title"].apply(clean_text)


torch.Size([1978, 512])
Validation Loss: 0.0000
Validation Accuracy: 1.0000
Test Accuracy: 1.0000
